In [ ]:
import re
import yaml
import pandas as pd

In [ ]:
specs_datacenter = pd.read_csv('./specs EFRA datacenter.csv')
specs_datacenter

In [ ]:
list_nodes = []
for name, vcpu, mem, num_gpu, model_gpu in zip(specs_datacenter['sn'],
                                               specs_datacenter['cpu_milli'],
                                               specs_datacenter['memory_mib'],
                                               specs_datacenter['gpu'],
                                               specs_datacenter['model']) :   
    name_node = name
    gpu_model = model_gpu
    gpu_count = str(num_gpu)
    gpu_milli = str(num_gpu * 1000)
    num_cpu = str(vcpu) + "m"
    memory = str(mem) + "Mi"


    resources = {
        "cpu": num_cpu,
        "memory": memory,
        "pods": "1001",
    }


    node = {
        "apiVersion": "v1",
        "kind": "Node",
        "metadata": {
            "labels": {
                "beta.kubernetes.io/os": "linux",
                "kubernetes.io/hostname": name_node,
                "kubernetes.io/os": "linux",
            },
            "name": name_node,
        },
        "status": {
            "allocatable": resources,
            "capacity": resources,  # identical to allocatable
        },
    }

    
    if num_gpu :
        resources["alibabacloud.com/gpu-count"] = gpu_count
        resources["alibabacloud.com/gpu-milli"] = gpu_milli
        node["metadata"]["labels"]["alibabacloud.com/gpu-card-model"] = gpu_model

    
    list_nodes.append(node)

In [ ]:
class NoAliasDumper(yaml.SafeDumper):
    def ignore_aliases(self, data):
        return True

# Dump the list of dictionaries into a single YAML.
text = yaml.dump_all(list_nodes, Dumper = NoAliasDumper)

# Add one blank line above and below every line that is exactly '---', thus using the same style used by the simulator's authors.
text = re.sub(r'(?m)^(---)$', r'\n\1\n', text)

with open("openb_node_list_gpu_node.yaml", "w") as f:
    f.write(text)